In [51]:
#!g1.1
%pip install --upgrade gdown
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14763 sha256=9be7727f107ba75d3da537d83a6b90c8466603909003f9d3ef1ede1d9e92d531
  Stored in directory: /tmp/xdg_cache/pip/wheels/7b/7b/5d/656f46cd6889e4c93977be9586901d0adc1271b2d876c84c96
Successfully built gdown
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [52]:
#!g1.1
import gdown

url = 'https://drive.google.com/drive/folders/1qw05BnA1O-XDgJ50OgNGFSlTa9Kls00j?usp=sharing'
gdown.download_folder(url, quiet=True)

['/home/jupyter/work/resources/aapd_test.tsv',
 '/home/jupyter/work/resources/aapd_train.tsv',
 '/home/jupyter/work/resources/aapd_validation.tsv',
 '/home/jupyter/work/resources/label_test',
 '/home/jupyter/work/resources/label_train',
 '/home/jupyter/work/resources/label_val',
 '/home/jupyter/work/resources/text_test',
 '/home/jupyter/work/resources/text_train',
 '/home/jupyter/work/resources/text_val']

In [53]:
#!g1.1
!wget https://gist.githubusercontent.com/ArseniyBolotin/7623835da1631b00fb150bcd5b0d909f/raw/wandb_writer.py -O wandb_writer.py

--2022-04-09 13:07:39--  https://gist.githubusercontent.com/ArseniyBolotin/7623835da1631b00fb150bcd5b0d909f/raw/wandb_writer.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2542 (2.5K) [text/plain]
Saving to: ‘wandb_writer.py’

wandb_writer.py     100%[===================>]   2.48K  --.-KB/s    in 0s      

2022-04-09 13:07:40 (42.9 MB/s) - ‘wandb_writer.py’ saved [2542/2542]



In [54]:
#!g1.1
# i need this one
def apply_to_dict_values(dict, f):
    for key, value in dict.items():
        dict[key] = f(value)

In [55]:
#!g1.1
BERT_TYPE = 'bert-base-uncased'

In [56]:
#!g1.1
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig

class AAPDDataset(Dataset):
    """AAPD dataset."""

    def __init__(self, path):
        self.path = path
        self.data = pd.read_csv(self.path, sep='\t', header=None)
        self.tokenizer = BertTokenizer.from_pretrained(BERT_TYPE)

    def __len__(self):
        return self.data.shape[0]

    @staticmethod
    def target_to_tensor(target):
        return torch.tensor([float(label) for label in target])

    def __getitem__(self, idx):
        data = self.tokenizer(self.data.iloc[idx, 1], return_tensors="pt", max_length=512, padding="max_length", truncation=True) # max_len=512 !DocBERT
        apply_to_dict_values(data, lambda x: x.flatten())
        return data, AAPDDataset.target_to_tensor(self.data.iloc[idx, 0])

In [58]:
#!g1.1
train_dataset = AAPDDataset('./aapd_train.tsv')
val_dataset = AAPDDataset('./aapd_validation.tsv')
test_dataset = AAPDDataset('./aapd_test.tsv')

In [73]:
#!g1.1
BATCH_SIZE = 16 # !DocBERT
N_CLASSES = test_dataset[0][1].shape[0]
BATCH_SIZE, N_CLASSES

(16, 54)

In [74]:
#!g1.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [75]:
#!g1.1
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [62]:
#!g1.1
config = BertConfig.from_pretrained(BERT_TYPE)
config.return_dict = True
bert = BertModel.from_pretrained(BERT_TYPE, config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
#!g1.1
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        # if name.find("layer.0") == -1: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    # print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(bert)

ModuleNotFoundError: No module named 'prettytable'

In [64]:
#!g1.1
import torch.nn as nn

class MultiLabelBert(nn.Module):
    def __init__(self, bert_model, n_classes, freeze_bert_weights=False):
        super(MultiLabelBert, self).__init__()
        self.bert_model = bert_model
        if freeze_bert_weights: # == false !DocBERT
            for param in self.bert_model.parameters():
                param.requires_grad = False
        self.n_bert_features = bert_model.pooler.dense.out_features
        self.n_classes = n_classes
        self.dense = nn.Linear(self.n_bert_features, self.n_classes)

    def forward(self, inputs):
        bert_output = self.bert_model(**inputs)
        return self.dense(bert_output.pooler_output)


In [65]:
#!g1.1
model = MultiLabelBert(bert, N_CLASSES).to(device)

In [66]:
#!g1.1
import torch.optim as optim

LEARNING_RATE = 2e-5 # !DocBERT

In [67]:
#!g1.1
%pip install wandb
from wandb_writer import WandbWriter

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.8 MB 1.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 144 kB 56.4 MB/s            
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8785 sha256=a294b60d3d46d7519db1aedde9b2943d795470fd7e84083c690ac02d4e34658f
  Stored in directory: /tmp/xdg_cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [68]:
#!g1.1
from sklearn.metrics import hamming_loss

In [69]:
#!g1.1
import tqdm
from sklearn.metrics import f1_score, hamming_loss

def train_model(
        model,
        train_dataloader,
        val_dataloader,
        optimizer,
        criterion,
        n_epochs,
        wandb_writer,
        wandb_iter_start = 0):

    wandb_iter = wandb_iter_start
    for epoch in range(1, n_epochs+1):
        model.train()
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(train_dataloader, f"Train epoch#{epoch}", leave=False):

            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets.to(device)
            logits = model(batch)
            all_targets.extend(targets.to('cpu').tolist())
            all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
            optimizer.zero_grad()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            wandb_writer.set_step(wandb_iter)
            wandb_writer.add_scalar("Batch train loss", loss.item())
            wandb_iter += 1

        wandb_writer.add_scalar("Train loss", total_loss / len(train_dataloader))
        wandb_writer.add_scalar("Train F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        wandb_writer.add_scalar("Train Hamming loss",(hamming_loss(all_targets, all_preds)))


        model.eval()

        with torch.no_grad():
            total_loss = 0.
            all_targets = []
            all_preds = []
            for batch, targets in tqdm.tqdm(val_dataloader, f"Val epoch#{epoch}", leave=False):
                apply_to_dict_values(batch, lambda x: x.to(device))
                targets = targets.to(device)
                logits = model(batch)
                all_targets.extend(targets.to('cpu').tolist())
                all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
                loss = criterion(logits, targets)
                total_loss += loss.item()
            
            wandb_writer.add_scalar("Validation loss", total_loss / len(val_dataloader))
            wandb_writer.add_scalar("Validation F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
            wandb_writer.add_scalar("Validation Hamming loss",(hamming_loss(all_targets, all_preds)))
    print(wandb_iter)


In [78]:
#!g1.1
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 7.75 Gb


In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats('cuda')
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

In [76]:
#!g1.1
model = MultiLabelBert(bert, N_CLASSES).to(device)
optimizer = optim.AdamW(params=[p for p in model.parameters() if p.requires_grad], lr=LEARNING_RATE, weight_decay=0.01)

In [79]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelDocBert"), wandb_iter_start=0)

Batch train loss,█▄▄▄▄▃▄▂▂▃▂▁▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▂▂▂▂▂▂▃▂▂▂▂▂▂
Train F1 (micro),▁
Train Hamming loss,▁
Train loss,▁
Validation F1 (micro),▁
Validation Hamming loss,▁
Validation loss,▁
steps_per_sec,▅▃▇▁▄▅▅▄▅▄▄▅▁▄▆▄▇▇▃▅▄▅▇▅▆▁█▆█▄▄▅▅▆▆▆▅▃▅▅
Batch train loss,0.07049
Train F1 (micro),0.54665
Train Hamming loss,0.03031



33650


In [82]:
#!g1.1
torch.save(model.state_dict(), "DocBERT-base-10.pt")

In [83]:
#!g1.1
torch.save(optimizer.state_dict(), "DocBERT-base-10-opt.pt")

In [87]:
#!g1.1
model.load_state_dict(torch.load("DocBERT-base-10.pt"))

<All keys matched successfully>

In [88]:
#!g1.1
optimizer.load_state_dict(torch.load("DocBERT-base-10-opt.pt"))

In [89]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelDocBert"), wandb_iter_start=33650)

Batch train loss,▆▂▁▃▃▄▄▇▆▅▅▆▅▇▅▃▅▄█
steps_per_sec,▁██████████▇████▇▇█
Batch train loss,0.02509
steps_per_sec,1.5931



67300


In [91]:
#!g1.1
torch.save(model.state_dict(), "DocBERT-base-20.pt")
torch.save(optimizer.state_dict(), "DocBERT-base-20-opt.pt")

In [ ]:
!wget https://gist.githubusercontent.com/ArseniyBolotin/4187ea4be63cc9ef2944e07692ed8f89/raw/google_drive.py -O google_drive.py

In [ ]:
import google_drive
google_drive.auth()

In [ ]:
model_name = "BERT_AAPD_20.pt"
torch.save(model, model_name)

In [ ]:
google_drive.save_model(model_name)

In [ ]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelDocBert"), wandb_iter_start=67300)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: c3n34ka (use `wandb login --relogin` to force relogin)


Train epoch#1:   1%|▏         | 45/3365 [00:25<30:20,  1.82it/s] 

In [ ]:
model_name = "BERT_AAPD_30.pt"
torch.save(model, model_name)
google_drive.save_model(model_name)

In [ ]:
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelBert"), wandb_iter_start=25249)

In [ ]:
model_name = "BERT_AAPD_40.pt"
torch.save(model, model_name)
google_drive.save_model(model_name)

In [ ]:
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 20, WandbWriter("MultiLabelBert"), wandb_iter_start=25249 + 8420)

In [ ]:
model_name = "BERT_AAPD_60.pt"
torch.save(model, model_name)
google_drive.save_model(model_name)